In [75]:
import pymysql
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
import tkinter as tk
import sqlite3
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


#connection for phpmyadmin
def connection():
    conn = pymysql.connect(
        host='localhost',
        user='root', 
        password='',
        db='inventory_db',
    )
    return conn

def refreshTable():
    for data in my_tree.get_children():
        my_tree.delete(data)

    for array in read():
        my_tree.insert(parent='', index='end', iid=array, text="", values=(array), tag="orow")

    my_tree.tag_configure('orow', background='#EEEEEE', font=('Arial', 12))
    my_tree.grid(row=3, column=4, columnspan=4, rowspan=5, padx=10, pady=10)

root = Tk()
root.title("Inventory Control System")
root.geometry("1080x600")
my_tree = ttk.Treeview(root)

#placeholders for entry
ph1 = tk.StringVar()
ph2 = tk.StringVar()
ph3 = tk.StringVar()
ph4 = tk.StringVar()
ph5 = tk.StringVar()

#placeholder set value function
def setph(word,num):
    if num ==1:
        ph1.set(word)
    if num ==2:
        ph2.set(word)
    if num ==3:
        ph3.set(word)
    if num ==4:
        ph4.set(word)
    if num ==5:
        ph5.set(word)


def graph():
    root=tkinter.Tk()
    root.wm_title('Embedding in tkinter')


    conn = connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM stock")
    df = pd.DataFrame(cursor.fetchall(), columns = ['product_id', 'product_name', 'price','qty'])

    fig = Figure(figsize=(5,4),dpi=100)
    fig.add_subplot(111).plot(df['product_id'],df['qty'])
    fig.suptitle("Total Inventory and balance Stock")
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tkinter.TOP,fill=tkinter.BOTH, expand=1)

    tkinter.mainloop()
       

def read():
    conn = connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM stock")
    results = cursor.fetchall()
    conn.commit()
    conn.close()
    return results

def add():
    id = str(idEntry.get())
    ds = str(dsEntry.get())
    price = str(priceEntry.get())
    qty = str(qtyEntry.get())
    

    if (id == "" or id == " ") or (ds == "" or ds == " ") or (price == "" or price == " ") or (qty == "" or qty == " "):
        messagebox.showinfo("Error", "Please fill up the blank entry")
        return
    else:
        try:
            conn = connection()
            cursor = conn.cursor()
            cursor.execute("INSERT INTO stock VALUES ('"+id+"','"+ds+"','"+price+"','"+qty+"') ")
            conn.commit()
            conn.close()
        except:
            messagebox.showinfo("Error", "Stud ID already exist")
            return

    refreshTable()
    

def reset():
    decision = messagebox.askquestion("Warning!!", "Delete all data?")
    if decision != "yes":
        return 
    else:
        try:
            conn = connection()
            cursor = conn.cursor()
            cursor.execute("DELETE FROM stock")
            conn.commit()
            conn.close()
        except:
            messagebox.showinfo("Error", "Sorry an error occured")
            return

        refreshTable()

def delete():
    decision = messagebox.askquestion("Warning!!", "Delete the selected data?")
    if decision != "yes":
        return 
    else:
        selected_item = my_tree.selection()[0]
        deleteData = str(my_tree.item(selected_item)['values'][0])
        try:
            conn = connection()
            cursor = conn.cursor()
            cursor.execute("DELETE FROM stock WHERE ID='"+str(deleteData)+"'")
            conn.commit()
            conn.close()
        except:
            messagebox.showinfo("Error", "Sorry an error occured")
            return

        refreshTable()

def select():
    try:
        selected_item = my_tree.selection()[0]
        id = str(my_tree.item(selected_item)['values'][0])
        ds = str(my_tree.item(selected_item)['values'][1])
        price = str(my_tree.item(selected_item)['values'][2])
        qty = str(my_tree.item(selected_item)['values'][3])

        setph(id,1)
        setph(ds,2)
        setph(price,3)
        setph(qty,4)
  
    except:
        messagebox.showinfo("Error", "Please select a data row")

def clear():

        id = str("")
        ds = str("")
        price = str("")
        qty = str("")

        setph(id,1)
        setph(ds,2)
        setph(price,3)
        setph(qty,4)

def update():
    selectedid = ""

    try:
        selected_item = my_tree.selection()[0]
        selectedid = str(my_tree.item(selected_item)['values'][0])
    except:
        messagebox.showinfo("Error", "Please select a data row")

    id = str(idEntry.get())
    ds = str(dsEntry.get())
    price = str(priceEntry.get())
    qty = str(qtyEntry.get())

    if (id == "" or id == " ") or (ds == "" or ds == " ") or (price == "" or price == " ") or (qty == "" or qty == " "):
        messagebox.showinfo("Error", "Please fill up the blank entry")
        return
    else:
        try:
            conn = connection()
            cursor = conn.cursor()
            cursor.execute("UPDATE stock SET ID='"+
            id+"', DESCRIPTION='"+
            ds+"', PRICE='"+
            price+"', QUANTITY='"+
            qty+"' WHERE ID='"+
            selectedid+"' ")
            conn.commit()
            conn.close()
        except:
            messagebox.showinfo("Error", "Stud ID already exist")
            return

    refreshTable()

label = Label(root, text="Inventory management System",fg = "white", bg="black",font=('Arial Bold', 30))
label.grid(row=0, column=0, columnspan=8, rowspan=2, padx=50, pady=40)

idLabel = Label(root, text="Product ID", font=('Arial', 15))
dsLabel = Label(root, text="Dsscription", font=('Arial', 15))
priceLabel = Label(root, text="Price", font=('Arial', 15))
qtyLabel = Label(root, text="Quantity", font=('Arial', 15))


idLabel.grid(row=3, column=0, padx=10, pady=0)
dsLabel.grid(row=4, column=0, padx=10, pady=0)
priceLabel.grid(row=5, column=0, padx=10, pady=0)
qtyLabel.grid(row=6, column=0, padx=10, pady=0)


idEntry = Entry(root, width=25, bd=5, font=('Arial', 15), textvariable = ph1)
dsEntry = Entry(root, width=25, bd=5, font=('Arial', 15), textvariable = ph2)
priceEntry = Entry(root, width=25, bd=5, font=('Arial', 15), textvariable = ph3)
qtyEntry = Entry(root, width=25, bd=5, font=('Arial', 15), textvariable = ph4)


idEntry.grid(row=3, column=1, columnspan=3, padx=5, pady=0)
dsEntry.grid(row=4, column=1, columnspan=3, padx=5, pady=0)
priceEntry.grid(row=5, column=1, columnspan=3, padx=5, pady=0)
qtyEntry.grid(row=6, column=1, columnspan=3, padx=5, pady=0)


addBtn = Button(
    root, text="Add", padx=5, pady=5, width=7,
    bd=3, font=('Arial', 15), bg="#84F894", command=add)
updateBtn = Button(
    root, text="Update", padx=5, pady=5, width=7,
    bd=3, font=('Arial', 15), bg="#84E8F8", command=update)
clearBtn = Button(
    root, text="Clear", padx=5, pady=5, width=7,
    bd=3, font=('Arial', 15), bg="#F4FE82", command=clear)
deleteBtn = Button(
    root, text="Delete", padx=15, pady=12, width=10,
    bd=5, font=('Arial', 15), bg="#FF9999", command=delete)
resetBtn = Button(
    root, text="Reset", padx=15, pady=12, width=10,
    bd=5, font=('Arial', 15), bg="#F398FF", command=reset)
selectBtn = Button(
    root, text="Select", padx=15, pady=12, width=10,
    bd=5, font=('Arial', 15), bg="#B5EAD7", command=select)
graphBtn = Button(
    root, text="Graph", padx=15, pady=15, width=47,
    bd=5, font=('Arial', 15), bg="#EEEEEE", command=graph)



addBtn.grid(row=7, column=1, columnspan=1, rowspan=1)
updateBtn.grid(row=7, column=2, columnspan=1, rowspan=1)
clearBtn.grid(row=7, column=3, columnspan=1, rowspan=1)
deleteBtn.grid(row=8, column=5, columnspan=1, rowspan=1)
resetBtn.grid(row=8, column=6, columnspan=1, rowspan=1)
selectBtn.grid(row=8, column=7, columnspan=1, rowspan=1)
graphBtn.grid(row=9, column=5, columnspan=3, rowspan=3)



style = ttk.Style()
style.configure("Treeview.Heading", font=('Arial Bold', 15))

my_tree['columns'] = ("Product ID","Description","Price","Quantity")

my_tree.column("#0", width=0, stretch=NO)
my_tree.column("Product ID", anchor=W, width=170)
my_tree.column("Description", anchor=W, width=150)
my_tree.column("Price", anchor=W, width=150)
my_tree.column("Quantity", anchor=W, width=165)


my_tree.heading("Product ID", text="Product ID", anchor=W)
my_tree.heading("Description", text="Description", anchor=W)
my_tree.heading("Price", text="Price", anchor=W)
my_tree.heading("Quantity", text="Quantity", anchor=W)


refreshTable()

root.mainloop()